# Sustainability Goals

Get the Sustainability Goals data from the World Bank Website:https://api.worldbank.org/v2/sources/46/indicators

Documentation on the Sustainability Goals can be found here: https://datatopics.worldbank.org/sdgatlas/




In [53]:
# Importing necessary packages
import pandas as pd
import numpy as np
import datetime
import wbdata
import plotly.express as px
import plotly.io as pio

In [49]:
# Requesting the indicators for the topic Global Sustainability Goals.  The 46 is the key for the Global Sustainability Goals Data.
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=46")
indicatorsJSON = indicators.json()

In [50]:
# Get all Global Sustainability indicators so we know what is available
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=46&per_page=500")
indicatorsJSON = indicators.json()
#print(indicatorsJSON) # to view ALL of the indicators as is, remove the "#" at the beginning of the print command

# Parse through the response to see the Indicator IDs and Names
for i in indicatorsJSON[1]:
    IDSindicators = (i["id"],i["name"])
    print(IDSindicators) # to view the indicator ids and names, remove the "#" at the beginning of the print command

('AG.LND.FRST.K2', 'Forest area (sq. km)')
('AG.LND.FRST.ZS', 'Forest area (% of land area)')
('AG.YLD.CREL.KG', 'Cereal yield (kg per hectare)')
('BN.CAB.XOKA.GD.ZS', 'Current account balance (% of GDP)')
('BN.KLT.PTXL.CD', 'Portfolio Investment, net (BoP, current US$)')
('BX.KLT.DINV.CD.WD', 'Foreign direct investment, net inflows (BoP, current US$)')
('BX.KLT.DINV.WD.GD.ZS', 'Foreign direct investment, net inflows (% of GDP)')
('BX.TRF.PWKR.DT.GD.ZS', 'Personal remittances, received (% of GDP)')
('DT.DOD.DECT.GN.ZS', 'External debt stocks (% of GNI)')
('DT.ODA.ALLD.CD', 'Net official development assistance and official aid received (current US$)')
('DT.ODA.ODAT.CD', 'Net official development assistance received (current US$)')
('DT.ODA.ODAT.CD1', 'Net official development assistance received (current US$)')
('DT.ODA.ODAT.KD', 'Net official development assistance received (constant 2018 US$)')
('DT.TDS.DPPF.XP.ZS', 'Debt service (PPG and IMF only, % of exports of goods, services and 

In [51]:
# Use the indicator code to define the "indicator" variable.  We can change this to get information on each of the indicators.
indicator = "EN.ATM.CO2E.PC"

# Parse through the response to get the "sourceNote" or definition for the desired indicator
for dict_entity in indicatorsJSON[1]:
    if dict_entity["id"] == indicator:
        print(dict_entity["sourceNote"])
    else:
        pass

Carbon dioxide emissions are those stemming from the burning of fossil fuels and the manufacture of cement. They include carbon dioxide produced during consumption of solid, liquid, and gas fuels and gas flaring.


In [63]:
#Get a list of all the location abbreviations so we know what they are for limiting selections.
dlocations = requests.get("http://api.worldbank.org/v2/sources/46/country?per_page=300&format=JSON")
dlocationsJSON = dlocations.json()

# Parse through the response to see the location IDs and names
dlocations = dlocationsJSON["source"][0]["concept"][0]["variable"]
listLen = int(len(dlocations))

# Create dataframe with location values
df = pd.DataFrame(columns=["id", "value"])     
for i in range(0,listLen):
    code = dlocations[i]["id"]
    name = dlocations[i]["value"]
    df = df.append({"id":code, "value":name}, ignore_index = True)
dlocationsList = df

# See first few items in the dataframe
print(dlocationsList.head(n=20))

     id                 value
0   ABW                 Aruba
1   AFG           Afghanistan
2   AGO                Angola
3   ALB               Albania
4   AND               Andorra
5   ARB            Arab World
6   ARE  United Arab Emirates
7   ARG             Argentina
8   ARM               Armenia
9   ASM        American Samoa
10  ATG   Antigua and Barbuda
11  AUS             Australia
12  AUT               Austria
13  AZE            Azerbaijan
14  BDI               Burundi
15  BEL               Belgium
16  BEN                 Benin
17  BFA          Burkina Faso
18  BGD            Bangladesh
19  BGR              Bulgaria


In [80]:
#Select the indicator, location, and time you would like for the API call
# Selecting the indicator
indicatorSelection = {"EN.ATM.CO2E.PC":"CO2 emissions (metric tons per capita)"}
# Select the countries or regions
locationSelection = ["ARE","AUS","BRA","BEL"]
# Selecting the time frame
timeSelection = (datetime.datetime(2010, 1, 1), datetime.datetime(2020, 12, 31))
#Set the source id number for the Sustainable Development Goals Data
source_id=46

In [81]:
# Make the API call and assign the resulting DataFrame to "CO2_data"
#Notes from wbdata: 
"""
    Convenience function to download a set of indicators and  merge them into a
        pandas DataFrame.  The index will be the same as if calls were made to
        get_data separately.
    :indicators: An dictionary where the keys are desired indicators and the
        values are the desired column names
    :country: a country code, sequence of country codes, or "all" (default)
    :data_date: the desired date as a datetime object or a 2-sequence with
        start and end dates
    :freq: the desired periodicity of the data, one of 'Y' (yearly), 'M'
        (monthly), or 'Q' (quarterly). The indicator may or may not support the
        specified frequency.
    :source: the specific source to retrieve data from (defaults on API to 2,
        World Development Indicators)
    :convert_date: if True, convert date field to a datetime.datetime object.
    :keep_levels: if True don't reduce the number of index levels returned if
        only getting one date or country
    :cache: use the cache
    :returns: a WBDataFrame
    """
CO2_data = wbdata.get_dataframe(indicatorSelection,
                            source=source_id,
                            country = locationSelection, 
                            data_date = timeSelection, 
                            convert_date = False)

In [82]:
CO2_data.head()

CO2 emissions (metric tons per capita)
country              date                                        
United Arab Emirates 2019                                     NaN
                     2018                               20.797498
                     2017                               21.950614
                     2016                               21.307610
                     2015                               20.948092